In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import gzip
import numpy as np
from scipy.optimize import minimize
from datetime import timedelta
import tensorflow as tf
from datetime import timedelta

In [3]:
df = pd.read_csv("/home/sathappan/workspace/time2event/hawkes/data/all_trades.csv")
df.t = pd.to_datetime(df.t)
pp = (df.t - df.t[0]).apply(lambda x: x/np.timedelta64(1, 'm')).value_counts()
events = []
e = 1.667e-6
for t, cnt in pp.iteritems():
    base_time = t
    for i in range(cnt):
        events.append(base_time + e*i)

events = np.array(sorted(events))
empirical = df.t
empirical_counts = empirical.value_counts().resample(rule='1min').sum()
events = events[None, :4000]


In [4]:
def calc_Ai(arrivals, mask, beta):
    tdiff = np.diff(arrivals) * mask[:, 1:]
    tdiff = np.exp(tdiff * -beta)
    A_i = np.zeros_like(arrivals, dtype=np.float128)
    for i in range(tdiff.shape[1]):
        #print(i,)
        A_i[:, i + 1] = tdiff[:, i] * (1 + A_i[:, i])
    return A_i, tdiff

In [5]:
mask = np.ones_like(events)
A_i, tempdiff = calc_Ai(events, mask, 1.0)

In [6]:
def calc_temporal_differences(arrivals, mask):
    temporal_diff = np.zeros((arrivals.shape[0], arrivals.shape[1], arrivals.shape[1]))
    #mask_tempdiff = np.zeros_like(temporal_diff)
    for t_i in range(arrivals.shape[1]):
        temporal_diff[:, :t_i, t_i] = arrivals[:, t_i] - arrivals[:, :t_i]
        #mask_tempdiff[:, :t_i, t_i] = 1.0
    mask_tempdiff = (temporal_diff != 0)
    
    return mask_tempdiff, temporal_diff
    ### Create a UsersxTimexTime matrix

In [7]:
mask_tdiff, temporal_diff = calc_temporal_differences(events, mask)

In [8]:
from scipy.sparse import csr_matrix

In [9]:
def loglikelihood():
    #tdiff = diff(arrivals) * mask[:, 1:]
    #A_i = tf.zeros_like(arrivals)
    #for i in range(tdiff.shape[0]):
    #    A_i[:, i + 1] = tdiff[:, i] * (1 + A[:, i ])
    #part1 = tf.log(mu + alpha * ais_tvar)
    #part1 = tf.reduce_sum(tf.log(mu + alpha * ais_tvar), axis=1)[:, None]
    hist_effects = tf.reduce_sum((tf.exp(-beta[:, :, None] * temp_diff_tvar))*mask_tdiff_tvar, axis=2)
    part1 = tf.reduce_sum(tf.log(mu + alpha * hist_effects), axis=1)
    #part1 = tf.reduce_sum()
    T_max = tf.reduce_max(arrivals_tvar*mask_tvar, axis=1)[:, None]
    part2 = mu * T_max
    tmp = tf.exp(-beta * ((T_max - arrivals_tvar)*mask_tvar))
    p3_tmp = tf.reduce_sum(tmp - tf.constant(1.0, dtype=tf.float64), axis=1)
    part3 = (alpha / beta)* p3_tmp[:, None]
    
    #return part1, part2, part3, part1 - part2 + part3
    return tf.reduce_sum(part1 - part2 + part3)

In [10]:
mask.shape

(1, 4000)

In [11]:
num_users = 1
alpha = tf.Variable(tf.random_uniform([num_users, 1], dtype=tf.float64), name='alpha')
#alpha = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), name='alpha')
beta = tf.Variable(tf.random_uniform([num_users, 1], dtype=tf.float64), name='beta')
#beta = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), name='beta')
#beta = tf.constant(1.0, dtype=tf.float64, name='beta')
mu = tf.Variable(tf.random_uniform([num_users, 1], dtype=tf.float64), name='mu')
lag_mult = tf.Variable(tf.zeros((4,1), dtype=tf.float64), name='lagrangian')
#mu = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), name='mu')
penalty = tf.constant(4, dtype=tf.float64)
learning_rate = tf.constant(0.00001, dtype=tf.float64)

In [12]:
T_max = 4000
arrivals_tvar = tf.placeholder(tf.float64, [None, T_max])
mask_tvar = tf.placeholder(tf.float64, [None, T_max])
temp_diff_tvar = tf.placeholder(tf.float64,[None, T_max, T_max])
mask_tdiff_tvar = tf.placeholder(tf.float64, [None, T_max, T_max])

In [13]:
def cost_func():
    nloglik = -loglikelihood()
    l = tf.reduce_sum(alpha)  + tf.reduce_sum(mu) + tf.reduce_sum(alpha - beta) 
    regularizer = tf.reduce_sum(tf.square(alpha)) + tf.reduce_sum(tf.square(beta)) + tf.reduce_sum(tf.square(mu))
    return nloglik + 0.5*regularizer

In [14]:
def ADMM_cost():
    nloglik = -loglikelihood()
    regularizer = (tf.reduce_sum(tf.square(alpha)) + tf.reduce_sum(tf.square(beta)) + 
                   tf.reduce_sum(tf.square(mu)) + tf.reduce_sum(tf.square(alpha - beta)))
    constraints = tf.stack([tf.reduce_sum(alpha), tf.reduce_sum(mu), tf.reduce_sum(alpha - beta), tf.reduce_sum(beta)], name='constraints', axis=0)
    lagrangian = nloglik + tf.reduce_sum(lag_mult[:,0]*constraints) + (penalty/2.0)*regularizer
    return lagrangian, constraints, regularizer
    

In [15]:
def ADMM_update():
    grad_alpha = optim.compute_gradients(lagrangian, var_list=[alpha])[0][0]
    alpha2 = alpha.assign(alpha - learning_rate * grad_alpha)
    
    grad_beta = optim.compute_gradients(lagrangian, var_list=[beta])[0][0]
    beta2 = beta.assign(beta - learning_rate * grad_beta)

    grad_mu = optim.compute_gradients(lagrangian, var_list=[mu])[0][0]
    mu2 = mu.assign(mu - learning_rate * grad_mu)
    
    lag2 = lag_mult.assign((lag_mult[:, 0] + penalty * (constraints))[:, None])
    return alpha2, beta2, mu2, lag2

In [16]:
#p3 = loglikelihood()
p3 = cost_func()
#grad = tf.gradients(p3, [alpha, beta, mu])
optim = tf.train.GradientDescentOptimizer(0.0001)#.minimize(p3)
lagrangian, constraints, regularizer = ADMM_cost()
al2, bt2, mu2, lag2 = ADMM_update()
ll = loglikelihood()


#optim = tf.train.AdamOptimizer()

#gr_alpha, gr_beta, gr_mu  = optim.compute_gradients()
#gr_alpha = (tf.clip_by_value(gr_alpha[0], 0.01, np.infty), gr_alpha[1])
#gr_beta = (tf.clip_by_value(gr_beta[0], 0.01, np.infty), gr_beta[1])
#gr_mu = (tf.clip_by_value(gr_mu[0], 0.01, np.infty), gr_mu[1])

#minmzr = optim.apply_gradients([gr_alpha,gr_beta, gr_mu])
#p1, p2, p3 = loglikelihood()
#capped_gvs = [(tf.clip_by_value(grad, 0, 1), var) for grad, var in gvs]
#train_op = optim.apply_gradients(capped_gvs)

In [17]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    #learning_rate = 0.0001
    
    sess.run(init)
    
    for i in range(300):
        #print(sess.run(constraints))
        #print(sess.run(lag_mult))
        #print(sess.run(ll))
        #print(i)
        #j2 = sess.run(optim.compute_gradients(lagrangian, var_list=[alpha]), feed_dict={arrivals_tvar: np.nan_to_num(events), mask_tvar:mask, temp_diff_tvar: temporal_diff, mask_tdiff_tvar:mask_tdiff}) 
        #j2 = sess.run(lag_mult[:, 0] - penalty*constraints, feed_dict={arrivals_tvar: np.nan_to_num(events), mask_tvar:mask, temp_diff_tvar: temporal_diff, mask_tdiff_tvar:mask_tdiff}) 
        #print(j2)
        #print(alpha.eval())
        j = sess.run([al2,bt2, mu2, lag2], feed_dict={arrivals_tvar: np.nan_to_num(events), mask_tvar:mask, temp_diff_tvar: temporal_diff, mask_tdiff_tvar:mask_tdiff})
        if i % 50 == 0:
            print(np.asscalar(j[0]), np.asscalar(j[1]), np.asscalar(j[2]))
            print(j[3][:, 0])
        #print(sess.run(loglikelihood()))
        #alpha.assign(alpha - learning_rate * gr_a)
        #j = sess.run(gr_b, feed_dict={arrivals_tvar: np.nan_to_num(events), mask_tvar:mask, temp_diff_tvar: temporal_diff, mask_tdiff_tvar:mask_tdiff})
        #beta.assign(beta - learning_rate * gr_b)
        #mu.assign(mu - learning_rate * gr_m)
        #lag_mult.assign(lag_mult - penalty * (constraints))



0.9405528609382989 0.6256697059784464 0.2624056148475916
[ 3.87395818  1.05551908  1.59496609  2.27899209]
0.8951771199841483 0.9827628916829314 0.2618217639754962
[ 177.08289643   52.89944463   -1.97943481  179.06233124]
0.9325151353408454 1.0247480096180717 0.2525714474523037
[ 361.2431181   104.7591663   -20.37714984  381.62026794]
0.8831469500440096 0.9613418784679577 0.21848232330454637
[ 544.29110865  152.34897165  -37.68691159  581.97802024]
0.7581892410783813 0.8117826041474162 0.16049378564227904
[ 709.80291262  190.73749849  -51.00864451  760.81155714]
0.579934742054835 0.6076901746403106 0.08332558079775985
[ 844.58972172  215.52620763  -59.10382471  903.69354643]


<tf.Tensor 'alpha/read:0' shape=(1, 1) dtype=float64>

In [17]:
events[:, -1]

array([ 406.00000167])